<div style="border-left: 6px solid #00A86B; color:white; padding:20px; border-radius:10px; font-family:Arial, sans-serif; text-align:center; font-size:28px; font-weight:bold;">
  🔁 03 – Transfer Learning and Model Selection
</div>

<div style="border-left: 6px solid #27ae60; color:white; margin-left:40px; padding:10px; border-radius:10px; font-family:Arial, sans-serif; font-size:24px;">
  <h2 style="margin-top: 0; font-size:24px;">📦 Import Libraries and Define Paths</h2>
</div>

<div style="margin-left:60px; padding:10px;"> 
  <p style="font-size:18px;">This is the initial block of the rare species image classification project.</p>

  <p>In this section, we perform the following tasks:</p>

  <ul style="line-height: 1.6;">
    <li>📁 <strong>Import libraries</strong> for data manipulation (<code>pandas</code>), file paths (<code>pathlib</code>), and image processing (<code>PIL</code>).</li>
    <li>🖼️ <strong>Apply visual styling</strong> using <code>matplotlib</code> and <code>seaborn</code> to ensure clean and consistent plots.</li>
    <li>📂 <strong>Define the main project directories</strong>, including image folders and the metadata CSV file.</li>
    <li>✅ <strong>Automatic path validation</strong> to ensure all required files and directories exist.</li>
  </ul>

  <p>This setup provides a reliable foundation for safely loading and exploring the dataset.</p>
</div>


In [35]:
# ========================================== 📦 Importar bibliotecas essenciais ==========================================
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from tensorflow.keras.applications import EfficientNetB0, ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, CSVLogger
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from pathlib import Path

In [18]:
# ========================================== 📂 Definir caminhos principais do projeto ==========================================
PROJECT_ROOT = Path().resolve().parent

PROCESSED_DIR = PROJECT_ROOT / 'data' / 'processed'
MODELS_DIR = PROJECT_ROOT / 'models'
REPORTS_DIR = PROJECT_ROOT / 'reports'
OUTPUTS_DIR = PROJECT_ROOT / 'output'
LOGS_DIR = OUTPUTS_DIR / 'logs'
PREDICTIONS_DIR = OUTPUTS_DIR / 'predictions'
TRAIN_DIR = PROCESSED_DIR / 'train'
VAL_DIR = PROCESSED_DIR / 'val'
TEST_DIR = PROCESSED_DIR / 'test'

<div style="border-left: 6px solid #27ae60; color:white; margin-left:40px; padding:10px; border-radius:10px; font-family:Arial, sans-serif; font-size:24px;">
  <h2 style="margin-top: 0; font-size:24px;">📦 Define Parameters</h2>
</div>

<div style="margin-left:60px; padding:10px;"> 
  <p>In this section, we define the core parameters that will guide the training process of the model. These include the input image size, batch size, number of training epochs, and the directory structure of the dataset.</p>
  
  <p>Setting these values early ensures consistency across all steps and allows for easier adjustments when experimenting with different model architectures or datasets.</p>
</div>


In [19]:
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10

<div style="border-left: 6px solid #27ae60; color:white; margin-left:40px; padding:10px; border-radius:10px; font-family:Arial, sans-serif; font-size:24px;">
  <h2 style="margin-top: 0; font-size:24px;">📂 Load and Prepare the Dataset</h2>
</div>

<div style="margin-left:60px; padding:10px; font-family:Arial, sans-serif; font-size:16px;"> 
  <p>This section is responsible for loading the processed dataset and preparing it for training and evaluation.</p>

  <p>Using <code>ImageDataGenerator</code>, the images are normalized (pixel values scaled between 0 and 1), and loaded in batches directly from the respective folders for:</p>

  <ul style="line-height: 1.6;">
    <li><strong>🟢 Training set</strong> — used to update model weights during learning</li>
    <li><strong>🟠 Validation set</strong> — used to monitor generalization and prevent overfitting</li>
    <li><strong>🔵 Test set</strong> — used for final evaluation after training</li>
  </ul>

  <p>The dataset is expected to be organized in subfolders where each folder represents one class label.</p>
</div>


In [20]:
datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_directory(TRAIN_DIR, target_size=IMAGE_SIZE, batch_size=BATCH_SIZE, class_mode='categorical')
val_generator = datagen.flow_from_directory(VAL_DIR, target_size=IMAGE_SIZE, batch_size=BATCH_SIZE, class_mode='categorical')
test_generator = datagen.flow_from_directory(TEST_DIR, target_size=IMAGE_SIZE, batch_size=BATCH_SIZE, class_mode='categorical', shuffle=False)

NUM_CLASSES = train_generator.num_classes

Found 8627 images belonging to 202 classes.
Found 2157 images belonging to 202 classes.
Found 1199 images belonging to 202 classes.


<div style="border-left: 6px solid #27ae60; color:white; margin-left:40px; padding:10px; border-radius:10px; font-family:Arial, sans-serif; font-size:24px;">
  <h2 style="margin-top: 0; font-size:24px;">📦 Baseline EfficientNetB0 Model</h2>
</div>

<div style="margin-left:60px; padding:10px;"> 
  
</div>


In [21]:
def run_efficientnetb0_pipeline(train_gen, val_gen, test_gen, model_name="efficientnetb0_baseline", image_size=IMAGE_SIZE, epochs=EPOCHS):
    models_dir = MODELS_DIR
    logs_dir = LOGS_DIR
    predictions_dir = PREDICTIONS_DIR
    reports_dir = REPORTS_DIR
    figures_dir = reports_dir / "figures"
    for d in [models_dir, logs_dir, predictions_dir, figures_dir, reports_dir]:
        d.mkdir(parents=True, exist_ok=True)

    train_generator = train_gen
    val_generator = val_gen
    test_generator = test_gen

    num_classes = train_generator.num_classes

    base_model = EfficientNetB0(include_top=False, weights="imagenet", input_shape=(image_size[0], image_size[1], 3), pooling='avg')
    base_model.trainable = False

    model = Sequential([
        base_model,
        BatchNormalization(),
        Dense(256, activation='relu'),
        Dropout(0.4),
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')

    ])

    model.compile(optimizer=Adam(learning_rate=0.0005), loss="categorical_crossentropy", metrics=["accuracy"])

    early_stop = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True, verbose=1)
    reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, verbose=1)
    csv_logger = CSVLogger(logs_dir / f"{model_name}_training_log.csv", append=False)

    history = model.fit(train_generator, validation_data=val_generator, epochs=epochs, callbacks=[csv_logger, early_stop, reduce_lr])

    model_path = models_dir / f"{model_name}.h5"
    model_weights_path = models_dir / f"{model_name}.weights.h5"
    model.save(model_path)
    model.save_weights(model_weights_path)

    val_loss, val_acc = model.evaluate(val_generator)

    acc_fig_path = figures_dir / f"{model_name}_accuracy_plot.png"
    plt.figure(figsize=(8, 5))
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Val Accuracy')
    plt.title('Training vs Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(acc_fig_path)
    plt.close()

    predictions = model.predict(test_generator)
    predicted_classes = predictions.argmax(axis=1)
    true_classes = test_generator.classes
    class_indices = test_generator.class_indices
    inv_class_indices = {v: k for k, v in class_indices.items()}
    predicted_labels = [inv_class_indices[i] for i in predicted_classes]
    true_labels = [inv_class_indices[i] for i in true_classes]

    report = classification_report(true_classes, predicted_classes, target_names=list(class_indices.keys()), output_dict=True)
    report_df = pd.DataFrame(report).transpose()
    report_path = reports_dir / f"{model_name}_classification_report.csv"
    report_df.to_csv(report_path)

    heatmap_path = figures_dir / f"{model_name}_classification_report_heatmap_top20.png"
    filtered_df = report_df.drop(["accuracy", "macro avg", "weighted avg"], errors="ignore")
    top_20 = filtered_df.sort_values("support", ascending=False).head(20)
    plt.figure(figsize=(10, 8))
    sns.heatmap(top_20[["precision", "recall", "f1-score"]], annot=True, fmt=".2f", cmap="YlGnBu", linewidths=0.5, annot_kws={"size": 9})
    plt.title("Top 20 Classes – Classification Report", fontsize=14)
    plt.xlabel("Metrics", fontsize=12)
    plt.ylabel("Class", fontsize=12)
    plt.tight_layout()
    plt.savefig(heatmap_path)
    plt.close()

    top_labels = list(top_20.index)
    label_to_index = {name: i for i, name in enumerate(class_indices.keys())}
    top_indices = [label_to_index[l] for l in top_labels]
    filtered_true = [i for i in true_classes if i in top_indices]
    filtered_pred = [p for i, p in enumerate(predicted_classes) if true_classes[i] in top_indices]
    cm = confusion_matrix(filtered_true, filtered_pred, labels=top_indices)
    cm_labels = [list(class_indices.keys())[i] for i in top_indices]
    fig, ax = plt.subplots(figsize=(12, 10))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=cm_labels)
    disp.plot(ax=ax, xticks_rotation=45, cmap='Blues', colorbar=True)
    plt.title("Confusion Matrix – Top 20 Classes", fontsize=14)
    plt.tight_layout()
    cm_path = figures_dir / f"{model_name}_confusion_matrix_top20.png"
    plt.savefig(cm_path)
    plt.close()

    cm = confusion_matrix(true_classes, predicted_classes)
    fig, ax = plt.subplots(figsize=(20, 20))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=list(class_indices.keys()))
    disp.plot(ax=ax, xticks_rotation='vertical', cmap='Blues')
    full_cm_path = figures_dir / f"{model_name}_confusion_matrix.png"
    plt.savefig(full_cm_path)
    plt.close()

    filenames = test_generator.filenames
    results_df = pd.DataFrame({
        "filename": filenames,
        "true_label": true_labels,
        "predicted_label": predicted_labels
    })
    pred_path = predictions_dir / f"{model_name}_predictions.csv"
    results_df.to_csv(pred_path, index=False)

    return {
        "model_path": model_path,
        "log_path": logs_dir / f"{model_name}_training_log.csv",
        "report_path": report_path,
        "heatmap_path": heatmap_path,
        "confusion_matrix": full_cm_path,
        "predictions_path": pred_path,
        "accuracy_plot": acc_fig_path,
        "val_accuracy": val_acc
    }

In [22]:
results_efficientnetb0 = run_efficientnetb0_pipeline(
    train_gen=train_generator,
    val_gen=val_generator,
    test_gen=test_generator,
    model_name="efficientnetb0_baseline"
)

print("📦 EfficientNetB0 – Results Summary:\n")
print(f"📁 Model saved at:              {results_efficientnetb0['model_path']}")
print(f"📄 Training log:                {results_efficientnetb0['log_path']}")
print(f"📊 Classification report (CSV): {results_efficientnetb0['report_path']}")
print(f"🧯 Report heatmap (Top 20):     {results_efficientnetb0['heatmap_path']}")
print(f"📉 Confusion matrix (full):     {results_efficientnetb0['confusion_matrix']}")
print(f"📈 Accuracy plot:               {results_efficientnetb0['accuracy_plot']}")
print(f"📑 Predictions CSV:             {results_efficientnetb0['predictions_path']}")
print(f"✅ Final validation accuracy:   {results_efficientnetb0['val_accuracy']:.2%}")


Epoch 1/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 71s 234ms/step - accuracy: 0.0149 - loss: 5.3083 - val_accuracy: 0.0185 - val_loss: 5.2623 - learning_rate: 5.0000e-04
Epoch 2/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 59s 219ms/step - accuracy: 0.0231 - loss: 5.1691 - val_accuracy: 0.0311 - val_loss: 5.2410 - learning_rate: 5.0000e-04
Epoch 3/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 61s 225ms/step - accuracy: 0.0213 - loss: 5.1286 - val_accuracy: 0.0250 - val_loss: 5.2080 - learning_rate: 5.0000e-04
Epoch 4/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 60s 223ms/step - accuracy: 0.0240 - loss: 5.1150 - val_accuracy: 0.0153 - val_loss: 5.1900 - learning_rate: 5.0000e-04
Epoch 5/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 62s 228ms/step - accuracy: 0.0226 - loss: 5.1171 - val_accuracy: 0.0283 - val_loss: 5.1697 - learning_rate: 5.0000e-04
Epoch 6/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 60s 221ms/step - accuracy: 0.0232 - loss: 5.0949 - val_accuracy: 0.0250 - val_loss: 5.2100 - learning_rate: 5.0000e-04
Epoch 7/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 59s 21

68/68 ━━━━━━━━━━━━━━━━━━━━ 12s 179ms/step - accuracy: 0.0206 - loss: 5.0425
38/38 ━━━━━━━━━━━━━━━━━━━━ 30s 734ms/step


c:\Users\Tiago Pedro\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Tiago Pedro\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Tiago Pedro\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _wa

📦 EfficientNetB0 – Results Summary:

📁 Model saved at:              D:\Repositories\DL_EOLP\models\efficientnetb0_baseline.h5
📄 Training log:                D:\Repositories\DL_EOLP\output\logs\efficientnetb0_baseline_training_log.csv
📊 Classification report (CSV): D:\Repositories\DL_EOLP\reports\efficientnetb0_baseline_classification_report.csv
🧯 Report heatmap (Top 20):     D:\Repositories\DL_EOLP\reports\figures\efficientnetb0_baseline_classification_report_heatmap_top20.png
📉 Confusion matrix (full):     D:\Repositories\DL_EOLP\reports\figures\efficientnetb0_baseline_confusion_matrix.png
📈 Accuracy plot:               D:\Repositories\DL_EOLP\reports\figures\efficientnetb0_baseline_accuracy_plot.png
📑 Predictions CSV:             D:\Repositories\DL_EOLP\output\predictions\efficientnetb0_baseline_predictions.csv
✅ Final validation accuracy:   2.50%


<div style="border-left: 6px solid #27ae60; color:white; margin-left:40px; padding:10px; border-radius:10px; font-family:Arial, sans-serif; font-size:24px;">
  <h2 style="margin-top: 0; font-size:24px;">📦 Baseline ResNet50 Model</h2>
</div>

<div style="margin-left:60px; padding:10px;"> 
  
</div>


In [36]:
def run_resnet50_pipeline(train_gen, val_gen, test_gen, model_name="resnet50_baseline", image_size=IMAGE_SIZE, epochs=EPOCHS):
    models_dir = MODELS_DIR
    logs_dir = LOGS_DIR
    predictions_dir = PREDICTIONS_DIR
    reports_dir = REPORTS_DIR
    figures_dir = reports_dir / "figures"
    for d in [models_dir, logs_dir, predictions_dir, figures_dir, reports_dir]:
        d.mkdir(parents=True, exist_ok=True)

    train_generator = train_gen
    val_generator = val_gen
    test_generator = test_gen

    num_classes = train_generator.num_classes

    base_model = ResNet50(include_top=False, weights="imagenet", input_shape=(image_size[0], image_size[1], 3), pooling='avg')
    base_model.trainable = False

    model = Sequential([
        base_model,
        BatchNormalization(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(256, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer=Adam(learning_rate=0.0005), loss="categorical_crossentropy", metrics=["accuracy"])

    early_stop = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True, verbose=1)
    reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, verbose=1)
    csv_logger = CSVLogger(logs_dir / f"{model_name}_training_log.csv", append=False)

    history = model.fit(train_generator, validation_data=val_generator, epochs=epochs, callbacks=[csv_logger, early_stop, reduce_lr])

    model_path = models_dir / f"{model_name}.h5"
    model_weights_path = models_dir / f"{model_name}.weights.h5"
    model.save(model_path)
    model.save_weights(model_weights_path)

    val_loss, val_acc = model.evaluate(val_generator)

    acc_fig_path = figures_dir / f"{model_name}_accuracy_plot.png"
    plt.figure(figsize=(8, 5))
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Val Accuracy')
    plt.title('Training vs Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(acc_fig_path)
    plt.close()

    predictions = model.predict(test_generator)
    predicted_classes = predictions.argmax(axis=1)
    true_classes = test_generator.classes
    class_indices = test_generator.class_indices
    inv_class_indices = {v: k for k, v in class_indices.items()}
    predicted_labels = [inv_class_indices[i] for i in predicted_classes]
    true_labels = [inv_class_indices[i] for i in true_classes]

    report = classification_report(true_classes, predicted_classes, target_names=list(class_indices.keys()), output_dict=True)
    report_df = pd.DataFrame(report).transpose()
    report_path = reports_dir / f"{model_name}_classification_report.csv"
    report_df.to_csv(report_path)

    heatmap_path = figures_dir / f"{model_name}_classification_report_heatmap_top20.png"
    filtered_df = report_df.drop(["accuracy", "macro avg", "weighted avg"], errors="ignore")
    top_20 = filtered_df.sort_values("support", ascending=False).head(20)
    plt.figure(figsize=(10, 8))
    sns.heatmap(top_20[["precision", "recall", "f1-score"]], annot=True, fmt=".2f", cmap="YlGnBu", linewidths=0.5, annot_kws={"size": 9})
    plt.title("Top 20 Classes – Classification Report", fontsize=14)
    plt.xlabel("Metrics", fontsize=12)
    plt.ylabel("Class", fontsize=12)
    plt.tight_layout()
    plt.savefig(heatmap_path)
    plt.close()

    top_labels = list(top_20.index)
    label_to_index = {name: i for i, name in enumerate(class_indices.keys())}
    top_indices = [label_to_index[l] for l in top_labels]
    filtered_true = [i for i in true_classes if i in top_indices]
    filtered_pred = [p for i, p in enumerate(predicted_classes) if true_classes[i] in top_indices]
    cm = confusion_matrix(filtered_true, filtered_pred, labels=top_indices)
    cm_labels = [list(class_indices.keys())[i] for i in top_indices]
    fig, ax = plt.subplots(figsize=(12, 10))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=cm_labels)
    disp.plot(ax=ax, xticks_rotation=45, cmap='Blues', colorbar=True)
    plt.title("Confusion Matrix – Top 20 Classes", fontsize=14)
    plt.tight_layout()
    cm_path = figures_dir / f"{model_name}_confusion_matrix_top20.png"
    plt.savefig(cm_path)
    plt.close()

    cm = confusion_matrix(true_classes, predicted_classes)
    fig, ax = plt.subplots(figsize=(20, 20))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=list(class_indices.keys()))
    disp.plot(ax=ax, xticks_rotation='vertical', cmap='Blues')
    full_cm_path = figures_dir / f"{model_name}_confusion_matrix.png"
    plt.savefig(full_cm_path)
    plt.close()

    filenames = test_generator.filenames
    results_df = pd.DataFrame({
        "filename": filenames,
        "true_label": true_labels,
        "predicted_label": predicted_labels
    })
    pred_path = predictions_dir / f"{model_name}_predictions.csv"
    results_df.to_csv(pred_path, index=False)

    return {
        "model_path": model_path,
        "log_path": logs_dir / f"{model_name}_training_log.csv",
        "report_path": report_path,
        "heatmap_path": heatmap_path,
        "confusion_matrix": full_cm_path,
        "predictions_path": pred_path,
        "accuracy_plot": acc_fig_path,
        "val_accuracy": val_acc
    }

In [37]:
results_resnet50 = run_resnet50_pipeline(
    train_gen=train_generator,
    val_gen=val_generator,
    test_gen=test_generator
)

print("📦 ResNet50 – Results Summary:\n")
print(f"📁 Model saved at:              {results_resnet50['model_path']}")
print(f"📄 Training log:                {results_resnet50['log_path']}")
print(f"📊 Classification report (CSV): {results_resnet50['report_path']}")
print(f"🧯 Report heatmap (Top 20):     {results_resnet50['heatmap_path']}")
print(f"📉 Confusion matrix (full):     {results_resnet50['confusion_matrix']}")
print(f"📈 Accuracy plot:               {results_resnet50['accuracy_plot']}")
print(f"📑 Predictions CSV:             {results_resnet50['predictions_path']}")
print(f"✅ Final validation accuracy:   {results_resnet50['val_accuracy']:.2%}")


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 8s 0us/step
Epoch 1/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 144s 513ms/step - accuracy: 0.0308 - loss: 5.1664 - val_accuracy: 0.0746 - val_loss: 5.0419 - learning_rate: 5.0000e-04
Epoch 2/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 136s 503ms/step - accuracy: 0.0731 - loss: 4.7508 - val_accuracy: 0.1052 - val_loss: 4.6924 - learning_rate: 5.0000e-04
Epoch 3/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 140s 520ms/step - accuracy: 0.0941 - loss: 4.5945 - val_accuracy: 0.1178 - val_loss: 4.4728 - learning_rate: 5.0000e-04
Epoch 4/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 138s 513ms/step - accuracy: 0.1129 - loss: 4.4361 - val_accuracy: 0.1307 - val_loss: 4.3732 - learning_rate: 5.0000e-04
Epoch 5/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 137s 509ms/step - accuracy: 0.1185 - loss: 4.3233 - val_accuracy: 0.1363 - val_loss: 4.2738 - learning_rate: 5.0000e-04
Epoch 6/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 140s 518ms/step - accuracy: 0.1453 - loss: 4.2066 - val_accuracy: 0.1437 - val_loss: 4.2100 - learning_rate: 

68/68 ━━━━━━━━━━━━━━━━━━━━ 27s 402ms/step - accuracy: 0.1961 - loss: 3.9513
38/38 ━━━━━━━━━━━━━━━━━━━━ 34s 824ms/step


c:\Users\Tiago Pedro\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Tiago Pedro\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Tiago Pedro\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _wa

📦 EfficientNetB0 – Results Summary:

📁 Model saved at:              D:\Repositories\DL_EOLP\models\resnet50_baseline.h5
📄 Training log:                D:\Repositories\DL_EOLP\output\logs\resnet50_baseline_training_log.csv
📊 Classification report (CSV): D:\Repositories\DL_EOLP\reports\resnet50_baseline_classification_report.csv
🧯 Report heatmap (Top 20):     D:\Repositories\DL_EOLP\reports\figures\resnet50_baseline_classification_report_heatmap_top20.png
📉 Confusion matrix (full):     D:\Repositories\DL_EOLP\reports\figures\resnet50_baseline_confusion_matrix.png
📈 Accuracy plot:               D:\Repositories\DL_EOLP\reports\figures\resnet50_baseline_accuracy_plot.png
📑 Predictions CSV:             D:\Repositories\DL_EOLP\output\predictions\resnet50_baseline_predictions.csv
✅ Final validation accuracy:   17.85%
